# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import sys
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

# coastsat modules
sys.path.insert(0, os.pardir)
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects, \
    NOC_tools, NOC_download

# directory where the data will be accessed and stored
dataPartition = 'c:\\data\\coastsat'
country = 'STV'
base_filepath = os.path.join(dataPartition, country, 'classification')  
sites_dir_path = os.path.join(os.path.join(base_filepath, 'training_sites'))
training_dir_path = os.path.join(os.path.join(base_filepath, 'data'))

### 1. Download images
For each site on which you want to train the classifier, save a .kml file with the region of interest (5 vertices clockwise, first and last points are the same, can be created from Google myMaps) in the folder \training_sites.

You only need a few images (~10) to train the classifier.

In [ ]:
roi_sites = os.listdir(sites_dir_path)

for roi_kml in roi_sites:
    
    sitename = roi_kml[:roi_kml.find('.')] 
    
    print('***********************************************************')
    print(f'processing roi: {sitename}')
    print()
    
    kml_filepath = os.path.join(sites_dir_path, roi_kml)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)

        
    all_dates = [['2020-01-01', '2020-01-31'],['2020-02-01', '2020-02-28'],['2020-03-01', '2020-03-31'],
                 ['2020-04-01', '2020-04-30'],['2020-05-01', '2020-05-31'],['2020-06-01', '2020-06-30'],
                 ['2020-07-01', '2020-07-31'],['2020-08-01', '2020-08-31'],['2020-09-01', '2020-09-30'],
                 ['2020-10-01', '2020-10-31'],['2020-11-01', '2020-11-30'],['2020-12-01', '2020-12-31']]

    all_dates = [['2019-01-01', '2020-12-31']]
                  
    for date_pair in all_dates:
        
        print(f'    from {date_pair[0]} to {date_pair[1]}')
        
        # put all the inputs into a dictionnary
        inputs = {'polygon': roi_polygon, 'dates': date_pair, 'sat_list': ['S2'],
                  'sitename': sitename, 'filepath': training_dir_path, 'include_T2': False}

        metadata = NOC_download.retrieve_training_images(inputs)

print('***********************************************************')
print(f'finished ...')
print('***********************************************************')